# Обработка фотографий покупателя

## Исследовательский анализ данных
### Описание данных
Сетевой супермаркет внедряет систему компьютерного зрения для обработки фотографий покупателей. Фотофиксация в прикассовой зоне поможет определять возраст клиентов, чтобы анализировать покупки и предлагать товары, которые могут заинтересовать покупателей этой возрастной группы и контролировать добросовестность кассиров при продаже алкоголя. Строится модель, которая по фотографии определит приблизительный возраст человека. В нашем распоряжении набор фотографий людей с указанием возраста.

**Цель проекта:** построить модель, которая будет предсказывать возраст покупателей, при этом `MAE`должно быть не более 8.

In [ ]:
# загрузка библиотек
import pandas as pd

from tensorflow.keras.applications.resnet import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# зададим параметры
optimizer = Adam(learning_rate=0.00005)
datagen = ImageDataGenerator(validation_split=0.25, rescale=1./255)

Датасет содержит 7591 изображение с соответствующими метками реального и кажущегося возраста. Общее число голосов составляет около 250 000. В среднем в датасете около 38 голосов за каждое изображение, и это делает средний видимый возраст очень стабильным (0,3 стандартной ошибки от среднего значения).



Обучение полностью проходило на графической карте (GPU) [Yandex Compute Cloud](https://cloud.yandex.ru/services/compute). В связи с этим, ниже будут только представлены функции, без выводов, которые использовались для подготовки данных и обучения модели.

## Подготовка данных и загрузка на сервер
Фотографии были дополнительно зумированы, перевёрнуты, а также изменён их масштаб для увеличения выборки и увеличения разнообразия фотографий, так как покупатели могут стоять близко или далеко от камеры у кассы, может быть разный угол наклона установки камер в разных магазинах, так и угол наклона головы также может различаться и тд. Таким образом, увеличиваем точность предстаказания нашей модели.

In [ ]:
# Загрузка обучающей выборки на сервер
def load_train(path):
    labels=pd.read_csv(path + "labels.csv")
    train_datagen = ImageDataGenerator(
        validation_split=0.25,
        rescale=1./255,
        zoom_range=[0.5, 1.5],
        horizontal_flip=True)

    train_gen_flow = train_datagen.flow_from_dataframe(
        dataframe=labels,
        directory=path + 'final_files/',
        x_col='file_name',
        y_col='real_age',
        target_size=(224, 224),
        batch_size=32,
        class_mode='raw',
        subset='training',
        seed=12345) 
    return train_gen_flow


In [ ]:
# Загрузка тестовой выборки на сервер
def load_test(path):
    labels=pd.read_csv(path + "labels.csv")
    test_datagen = ImageDataGenerator(
        validation_split=0.25,
        rescale=1./255)

    test_gen_flow = test_datagen.flow_from_dataframe(
        dataframe=labels,
        directory=path + 'final_files/',
        x_col='file_name',
        y_col='real_age',
        target_size=(224, 224),
        batch_size=32,
        class_mode='raw',
        subset='validation',        
        seed=12345)
    return test_gen_flow

## Обучение модели
Обучение данных происходило на модели `ResNet50`. Были выставлены аргументы:
* `weights` = 'imagenet' (от англ. «сеть изображений»), чтобы улучшить результат, так как будет "дообучение" на ImageNet
* `include_top` = True (англ. «добавить верхушку»), чтобы указать, что в конце архитектуры ResNet есть два слоя: GlobalAveragePooling2D и Dense.

In [ ]:
# Создание модели
def create_model(input_shape):
    backbone = ResNet50(input_shape=input_shape,
                    weights='imagenet', 
                    include_top=False)

    model = Sequential()
    model.add(backbone)
    model.add(GlobalAveragePooling2D())
    model.add(Dense(1, activation='relu')) 
    model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])
    return model

Наилучший результат у меня был достигнул на 40 эпохах:

In [ ]:
# Запуск модели (обучение)
def train_model(model, train_data, test_data, batch_size=None, epochs=40, 
                steps_per_epoch=None, validation_steps=None):
    model.fit(train_data, 
              validation_data=test_data,
              batch_size=batch_size, epochs=epochs,
              steps_per_epoch=steps_per_epoch,
              validation_steps=validation_steps,
              verbose=2, shuffle=True)
 
    return model

Ниже ход обучения модели. Для сокращения, оставлены были только 5 эпох:

    Train for 178 steps, validate for 60 steps

    Epoch 1/40
    178/178 - 113s - loss: 365.6234 - mae: 14.1019 - val_loss: 1051.1215 - val_mae: 27.7879

    Epoch 10/40
    178/178 - 90s - loss: 42.1746 - mae: 4.9861 - val_loss: 89.2430 - val_mae: 7.2008

    Epoch 20/40
    178/178 - 90s - loss: 23.1570 - mae: 3.6620 - val_loss: 71.1486 - val_mae: 6.1434

    Epoch 30/40
    178/178 - 87s - loss: 15.1969 - mae: 2.9909 - val_loss: 66.0796 - val_mae: 5.8567

    Epoch 40/40
    178/178 - 93s - loss: 11.0937 - mae: 2.5385 - val_loss: 60.3392 - val_mae: 5.7854

    60/60 - 10s - loss: 60.3392 - mae: 5.7854

**Test MAE: 5.7854**

`MAE` меньше 8, а значит цель данного проекта достигнута. Перейдём к выводам.

## Выводы

Была постоена и обучена свёрточная нейронная сеть на базе `ResNet50` из библиотеки `tensorflow.keras` на датасете с фотографиями людей разных возрастов для сетевого супермаркета. Нейронная сеть была обучена с целью фотофиксации в прикассовой зоне для определения возраста клиентов, чтобы анализировать покупки и предлагать товары, которые могут заинтересовать покупателей этой возрастной группы и контролировать добросовестность кассиров при продаже алкоголя.

По ТЗ модель `MAE`< 8, однако получилось достичь `MAE` = 5.7854, что считаю хорошим результатом.